# INSTALAR LIBRERIAS

In [1]:
# !pip install docx2pdf
#pip install ipywidgets

In [2]:
gegeg

NameError: name 'gegeg' is not defined

# IMPORTAR

In [ ]:
from docx2pdf import convert
import os
import time

# CONSTRUIR LA RUTA DE LOS ARCHIVOS

In [ ]:
def get_unidad_local():
    try:
        # ruta del archivo que contiene la información necesaria
        with open('../../../../unidad_local.txt', 'r') as file:
            unidad = file.readline().strip()
            return unidad
    except FileNotFoundError:
        print("El archivo 'unidad_local.txt' no se encuentra en la ruta especificada.")
    return None

def get_ruta_local():
    try:
        with open('../../../ruta_local.txt', 'r') as file:
            ruta = file.readline().strip()
            return ruta
    except FileNotFoundError:
        print("El archivo 'ruta_local.txt' no se encuentra en la ruta especificada.")
    return None

In [ ]:
unidad = get_unidad_local()
ruta = get_ruta_local()
ruta_completa = os.path.join(unidad, ruta)
ruta_completa

In [ ]:
# colocar aqui la ruta de los archivos
path = ruta_completa + r'/informes'
contenido = os.listdir(path)
rutas =[]
i = 0
for name in contenido:
    rutas.append(path + '/' + name)
    i+=1

print('SE ENCONTRARON ' + str(i) + ' INFORMES')
print(len(rutas), 'INFORMES')
rutas

# INICIA EL PROCESO DE CONVERCIÓN

In [ ]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

In [ ]:
rutas[1].replace('.docx','.pdf').replace('informes','pdf')

In [ ]:
i = 1
for filename in rutas:
    print('CONVIRTIENDO... INFORME ' + str(i) + ' ' + filename)
    # se realiza la convrsion pasando parametro de salida y entrada, 
    # para la salida se reemplaza la extencion y la ruta
    print(filename)
    print(filename.replace('.docx','.pdf').replace('informes','pdf'))
    pdf = filename.replace('.docx','.pdf').replace('informes','pdf').split('/')[-1]
    convert(filename, r'C:\Users\bismarksr\Downloads' + pdf)
    i+=1
    time.sleep(5)
print('END')

# Leer archivos

In [ ]:
# CONEXION A GOOGLE DRIVE
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
contenido = os.listdir(ruta_completa + r'/pdf')
dic_files = {}
for file in contenido:
    split = file.split('_')
    dic_files[int(split[0])] = file
print(len(dic_files), 'informes')
dic_files

# CARGAR LOS PDFS A GOOGLE DRIVE

In [3]:
# CARGAR LOS PDFS A GOOGLE DRIVE
dic_urls = {}
# View all folders and file in your Google Drive
fileList = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
for file in fileList:
  #print('Title: %s, ID: %s' % (file['title'], file['id']))
  # Get the folder ID that you want
  if(file['title'] == "2DA_INSPECCION"):
      fileID = file['id']

for file in dic_files:
    file1 = drive.CreateFile({"mimeType": "application/pdf", "parents": [{"kind": "drive#fileLink", "id": fileID}]})
    file1['title'] = dic_files[file]
    file1.SetContentFile(ruta_completa + r'/pdf//' + dic_files[file])
    file1.Upload() # Upload the file.
    print('Created file %s with mimeType %s' % (file1['title'], file1['mimeType']))
    permission = file1.InsertPermission({
                        'type': 'anyone',
                        'value': 'anyone',
                        'role': 'reader'})
    dic_urls[file] = file1['alternateLink']
dic_urls
print('end')

NameError: name 'drive' is not defined

In [ ]:
print(len(dic_urls), 'URLs')
dic_urls

# conexion a amigocloud

In [ ]:
import sys
sys.path.append('../../../_amigocloud')

from amigocloud import AmigoCloud

In [ ]:
# leer token
acceso = open(ruta_completa + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [ ]:
for key in dic_urls:
    query = {'query': 'update dataset_307560 set link_doc_2da = \'{link}\' where cast(SPLIT_PART(canhero, \' / \', 1) as integer) = \'{cod}\''.format(link=dic_urls[key], cod=key)}
    res = amigocloud.post('https://app.amigocloud.com/api/v1/projects/32672/sql', query)
    print(key)
print('end')